## Loading the Data

In [1]:
from datasets import load_from_disk

dataset = load_from_disk('rat-poc-ds-w-context')

In [2]:
dataset['train'][0]

{'Source': 'འཇིག་ལས་འདས་པའི་གང་འདུལ་ལོ།།',
 'Target': 'Taming with transcendent beings.',
 'File_Name': 'TM3076',
 'Machine Aligned': False,
 '__index_level_0__': 1176089,
 'Tag': 'Intrinsic Existence, Conventional Existence',
 'context': ['འགྲོ་ཀུན་སྒྲིབ་པ་གཉིས་སྤངས་ཏེ།\xa0། -> May all beings conquer the two obscurations',
  'དགེ་བས་མཁའ་མཉམ་ལུས་ཅན་མ་ལུས་པ།། ཐེག་མཆོག་གོ་གྱོན་ཤེས་རབ་མཚོན་ཐོགས་ནས།། བདུད་བཞིའི་དགྲ་སྡེ་མ་ལུས་ཀུན་བཅོམ་སྟེ།། སྐུ་གསུམ་ནོར་བུའི་ཁྲི་ལ་འཁོད་གྱུར་ཅིག། -> Through this virtue, may all embodied beings throughout space without exception, Put on the armor of the Supreme Vehicle and having raised the weapon of wisdom, May they overcome all without exception of the host of enemies which are the four demons And be set on the jeweled throne of the three bodies.',
  'སྐྱེ་འགག་ཡོད་མེད་ལ་སོགས་པའི་དམིགས་པ་དང་འཛིན་པའི་ཡུལ་ལས་འདས་པའི་རིག་སྟོང་སྤྲོས་བྲལ་མཉམ་པ་ཉིད་ཀྱི་ཁོར་ཡུག་ཡིན་ཏེ། -> Phenomena therefore transcend all objects of reference and clinging, such as origin and cessat

In [3]:
dataset['test'][0]

{'Source': ' དབང་ཤེས་ནི་རྟགས་ལས་དཔག་མི་དགོས་པར་མངོན་སུམ་དུ་ངེས་པའི་ཕྱིར་རོ།།',
 'Target': ' Sense cognitions need not infer from signs but can ascertain things directly.',
 'File_Name': 'TM0713',
 'Machine Aligned': False,
 '__index_level_0__': 168915,
 'Tag': 'Prophecies, Rituals',
 'context': ['འགྲོ་ཀུན་སྒྲིབ་པ་གཉིས་སྤངས་ཏེ།\xa0། -> May all beings conquer the two obscurations',
  'དགེ་བས་མཁའ་མཉམ་ལུས་ཅན་མ་ལུས་པ།། ཐེག་མཆོག་གོ་གྱོན་ཤེས་རབ་མཚོན་ཐོགས་ནས།། བདུད་བཞིའི་དགྲ་སྡེ་མ་ལུས་ཀུན་བཅོམ་སྟེ།། སྐུ་གསུམ་ནོར་བུའི་ཁྲི་ལ་འཁོད་གྱུར་ཅིག། -> Through this virtue, may all embodied beings throughout space without exception, Put on the armor of the Supreme Vehicle and having raised the weapon of wisdom, May they overcome all without exception of the host of enemies which are the four demons And be set on the jeweled throne of the three bodies.',
  'སྐྱེ་འགག་ཡོད་མེད་ལ་སོགས་པའི་དམིགས་པ་དང་འཛིན་པའི་ཡུལ་ལས་འདས་པའི་རིག་སྟོང་སྤྲོས་བྲལ་མཉམ་པ་ཉིད་ཀྱི་ཁོར་ཡུག་ཡིན་ཏེ། -> Phenomena therefore transcend all obj

## Load Tokenizer, Model, and Data Collator

In [4]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [5]:
# Generate a list of all Tibetan Unicode characters (U+0F00 to U+0FFF)
tibetan_chars = [chr(codepoint) for codepoint in range(0x0F00, 0x0FFF)]

# Add the Tibetan characters to the tokenizer's vocabulary
new_tokens = [char for char in tibetan_chars if char not in tokenizer.get_vocab()]

# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Resize model embeddings to accommodate the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

Embedding(32355, 512)

In [6]:
enc = tokenizer.encode(dataset['train'][0]['Source'])
dec = tokenizer.decode(enc)
dec

'འཇིག་ལས་འདས་པའི་གང་འདུལ་ལོ།།</s>'

## Preprocess Data

The dataset can now be tokenized for training.

In [7]:
def preprocess_with_single_context(example):
    # Use the first context if it exists, otherwise use a fallback
    context = example['context'][0] if example['context'] and len(example['context']) > 0 else ""
    # Create input text using the first context
    input_text = f"translate Tibetan to English: {example['Source']} Context: {context}"
    target_text = example['Target']

    # Tokenize input and target
    return {
        "input_ids": tokenizer(input_text, padding="max_length", truncation=True, max_length=512).input_ids,
        "labels": tokenizer(target_text, padding="max_length", truncation=True, max_length=512).input_ids,
    }

# Apply preprocessing
dataset_with_contexts = dataset.map(
    preprocess_with_single_context,
    batched=False,  # Process one example at a time
)



Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

## Define Metric

In [8]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    avg_gen_len = np.mean(prediction_lens)

    # Return rounded results
    return {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "gen_len": round(avg_gen_len, 4),
    }

## Train the Model

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"rat-poc-single-context",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_with_contexts['train'],
    eval_dataset=dataset_with_contexts['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/16875 [00:00<?, ?it/s]

  0%|          | 0/33750 [00:00<?, ?it/s]

{'loss': 0.6562, 'grad_norm': 0.2886229455471039, 'learning_rate': 0.0002955555555555555, 'epoch': 0.04}
{'loss': 0.5483, 'grad_norm': 0.29981422424316406, 'learning_rate': 0.0002911111111111111, 'epoch': 0.09}
{'loss': 0.4934, 'grad_norm': 0.20636606216430664, 'learning_rate': 0.0002866666666666667, 'epoch': 0.13}
{'loss': 0.5085, 'grad_norm': 0.23460017144680023, 'learning_rate': 0.00028222222222222223, 'epoch': 0.18}
{'loss': 0.4873, 'grad_norm': 0.34310856461524963, 'learning_rate': 0.0002777777777777778, 'epoch': 0.22}
{'loss': 0.4961, 'grad_norm': 0.11381765455007553, 'learning_rate': 0.00027333333333333333, 'epoch': 0.27}
{'loss': 0.4859, 'grad_norm': 0.20495426654815674, 'learning_rate': 0.0002688888888888889, 'epoch': 0.31}
{'loss': 0.4955, 'grad_norm': 0.2735089063644409, 'learning_rate': 0.00026444444444444443, 'epoch': 0.36}
{'loss': 0.4901, 'grad_norm': 0.4512806832790375, 'learning_rate': 0.00026, 'epoch': 0.4}
{'loss': 0.5071, 'grad_norm': 0.4021315276622772, 'learning_r

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

  0%|          | 0/625 [00:00<?, ?it/s]

{'eval_loss': 0.44861406087875366, 'eval_bleu': 0.0369, 'eval_chrf': 5.56, 'eval_gen_len': 17.274, 'eval_runtime': 183.7488, 'eval_samples_per_second': 27.211, 'eval_steps_per_second': 3.401, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'train_runtime': 10777.0717, 'train_samples_per_second': 12.527, 'train_steps_per_second': 3.132, 'train_loss': 0.46936164731626157, 'epoch': 3.0}


TrainOutput(global_step=33750, training_loss=0.46936164731626157, metrics={'train_runtime': 10777.0717, 'train_samples_per_second': 12.527, 'train_steps_per_second': 3.132, 'total_flos': 1.827114319872e+16, 'train_loss': 0.46936164731626157, 'epoch': 3.0})